In [14]:
# %load ../../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt


# load MATLAB files
from scipy.io import loadmat
from scipy.optimize import minimize
from sklearn.preprocessing import PolynomialFeatures
import sys
from sklearn.linear_model import LogisticRegression
 
#%config InlineBackend.figure_formats = {'pdf',}
#%matplotlib inline

import seaborn as sns

plt.close("all") # Clsoe all plots


### Loading data 

In [15]:
# Load data
data = loadmat('ex3data1.mat')
data.keys()

#Load the weights
weights = loadmat('ex3weights.mat')
weights.keys()

y = data['y']

# Add x0 
X = np.c_[np.ones((data['X'].shape[0],1)), data['X']]

m,n = X.shape
 
print('X: {} (with intercept)'.format(X.shape))
print('y: {}'.format(y.shape))

print('\nNumber of features (n): %.0f'%(n))
print('Number of training examples (nm): %.0f'%(m))

theta1, theta2 = weights['Theta1'], weights['Theta2']

print('\ntheta1: {}'.format(theta1.shape))
print('theta2: {}'.format(theta2.shape))

sample = np.random.choice(X.shape[0], 20)
print('\nSample:')
plt.imshow(X[sample,1:].reshape(-1,20).T)
plt.axis('off');

X: (5000, 401) (with intercept)
y: (5000, 1)

Number of features (n): 401
Number of training examples (nm): 5000

theta1: (25, 401)
theta2: (10, 26)

Sample:


## Multiclass Logistic Regression

regularisationc can reduce the weight of the parameters in anywa function by increaseing the respective cost.
we edit the hypothesis to inflate the cost and thus reduce the fitting parameters

Note: \theta_0 is not to be regularised adn is calcualted seperately when using the vertorised method

Regularized Cost Function
for $\theta = 1,2,3,...n$
$$ J(\theta) = -\frac{1}{m}\bigg[\sum_{i=1}^{m}\big[y^{(i)} log( h_\theta(x^{(i)}))+(1-y^{(i)})log(1-h_\theta(x^{(i)}))\big] + \frac{\lambda}{2m}\sum_{j = 1}^{n}\theta_{j}^{2}\bigg]$$


Vectorized Cost Function
$$ J(\theta) = -\frac{1}{m}\big((log(h)^Ty+(log(1-h)^T(1-y)\big) + \frac{\lambda}{2m}\sum_{j = 1}^{n}\theta_{j}^{2}$$

Gradient Descent: Repeat until converge<br>
for j = 0 (The intercept is not regularised)
$$\theta_0 : = \theta_0- \alpha\frac{\partial J(\theta)}{\partial\theta_{0}}$$
for j = 1,2,3...n
$$\theta_j : = \theta_j- \alpha\Big[\Big(\frac{\partial J(\theta)}{\partial\theta_{j}}\Big) + \frac{\lambda}{m}\theta_{j}\Big] $$


Partial derivatives:<br>
for j = 0 
$$ \frac{\partial J(\theta)}{\partial\theta_{0}} = \frac{1}{m}\sum_{i=1}^{m} ( h_\theta (x^{(i)})-y^{(i)})x^{(i)}_{0} $$

for j = 1,2,3...m 
$$ \frac{\partial J(\theta)}{\partial\theta_{j}} = \frac{1}{m}\sum_{i=1}^{m} ( h_\theta (x^{(i)})-y^{(i)})x^{(i)}_{j} +\frac{\lambda}{m}\theta_{j}^{2}$$

Vectorized Partial Derivative <br>
for j = 0 
$$\frac{\partial J(\theta)}{\partial\theta_{0}} = \frac{1}{m} X^T(h-y)$$
for j = 1,2,3... n
$$\frac{\partial J(\theta)}{\partial\theta_{j}} = \frac{1}{m} X^T(h-y) + \frac{\lambda}{m}\theta_{j}^{2}$$
where $h = g(X\theta)$

                                

### Regularised Cost Function

In [16]:
def sigmoid(z):
    return(1 / (1 + np.exp(-z)))
    
#logitic regression cost fucntion (Regularised)
def lrcostFunctionReg(theta, reg, X, y):
    m = y.size
    h = sigmoid(X.dot(theta))
    
    #cost fuinction
    J = -1 * (1/m) * (np.log(h).T.dot(y) + np.log(1 - h).T.dot(1 - y)) + (reg/(2 * m))*np.sum(np.square(theta[1:]))
    
    if np.isnan(J[0]):
        return(np.inf)
    return(J[0])    

# logistic regreession gradient of cost function
def lrgradientReg(theta, reg, X,y):
    m = y.size
    h = sigmoid(X.dot(theta.reshape(-1,1)))
     
    #Gradient
    grad = (1/m)*X.T.dot(h-y) + (reg/m)*np.r_[[[0]],theta[1:].reshape(-1,1)]
        
    return(grad.flatten())

### One - Vrs - All Prediction

In [18]:

# logistic regression - one - Vrs - all
def oneVsAll(features, classes, n_labels, reg):
    initial_theta = np.zeros((X.shape[1],1))  # 401x1
    all_theta = np.zeros((n_labels, X.shape[1])) #10x401

    for c in np.arange(1, n_labels + 1):
        res = minimize(lrcostFunctionReg, initial_theta, args = (reg, features, (classes == c) * 1), 
                       method = None,jac = lrgradientReg, options = {'maxiter':50})
        all_theta[c-1] = res.x
    return(all_theta)


theta = oneVsAll(X, y, 10, 0.1)
print(theta)


def predictOneVsAll(all_theta, features):
    probs = sigmoid(X.dot(all_theta.T))
        
    # Adding one because Python uses zero based indexing for the 10 columns (0-9),
    # while the 10 classes are numbered from 1 to 10.
    return(np.argmax(probs, axis = 1) + 1)
 
pred = predictOneVsAll(theta, X)

print('Training set accuracy with Obe-Vrs_all: {} %'.format(np.mean(pred == y.ravel())*100))

[[ -2.39142338e+00   0.00000000e+00   0.00000000e+00 ...,   6.90403614e-04
    1.25548916e-07   0.00000000e+00]
 [ -3.01137869e+00   0.00000000e+00   0.00000000e+00 ...,   1.48908371e-03
   -1.69495224e-04   0.00000000e+00]
 [ -4.51398334e+00   0.00000000e+00   0.00000000e+00 ...,  -1.60669585e-05
    3.43368479e-07   0.00000000e+00]
 ..., 
 [ -8.41428002e+00   0.00000000e+00   0.00000000e+00 ...,  -6.96539602e-05
    6.26414118e-06   0.00000000e+00]
 [ -5.13253825e+00   0.00000000e+00   0.00000000e+00 ...,  -1.54282889e-04
    5.92217956e-06   0.00000000e+00]
 [ -4.38685169e+00   0.00000000e+00   0.00000000e+00 ...,  -3.60354278e-04
    9.53842298e-06   0.00000000e+00]]
Training set accuracy with Obe-Vrs_all: 93.24 %


### Multiclass Logistic Regression with scikit-learn

In [12]:
clf = LogisticRegression(C = 10, penalty='l2', solver='liblinear')
# Scikit-learn fits intercept automatically, so we exclude first column with 'ones' from X when fitting.
clf.fit(X[:,1:],y.ravel())

pred2 = clf.predict(X[:,1:])
print('Training set accuracy with sklearn: {} %'.format(np.mean(pred2 == y.ravel())*100))


Training set accuracy with sklearn: 96.5 %


### Neural Networks

In [13]:
def predict(theta_1, theta_2, features):
    z2 = theta_1.dot(features.T)
    a2 = np.c_[np.ones((data['X'].shape[0],1)), sigmoid(z2).T]
    
    z3 = a2.dot(theta_2.T)
    a3 = sigmoid(z3)
        
    return(np.argmax(a3, axis=1)+1) 

pred = predict(theta1, theta2, X)
print('Training set accuracy: {} %'.format(np.mean(pred == y.ravel())*100))

Training set accuracy: 97.52 %
